[Link to different notebook](combine_scrapes.ipynb)

In [64]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [65]:
df_nlp = pd.read_csv('fin_cleaned_nlp.csv')

In [66]:
df_nlp.head()

,Unnamed: 0,Text,Clean
0,PDXorax,"The Lump of Labor doesn't apply in this case, ...",lump labor doesnt appli case look rd labor aut...
1,PrincePizza1,"I think things ""getting better"" to people like...",think thing get better peopl like everybodi ag...
2,Marmar79,Thank you. This made my dayHe so edgy.\n\nEdit...,thank made dayh edgi edit dont think wrong thi...
3,IRISHE3,I just wish politicians would put the money sh...,wish politician would put money shit real doll...
4,RonZiggy,Climate change as well.Wow...just wowWhat is t...,climat chang wellwowjust wowwhat fromwel happe...


In [67]:
df_nlp.set_index('Unnamed: 0',inplace=True)

In [68]:
df_nlp.dropna(axis=0,inplace=True)

In [69]:
df_nlp.head()

,Text,Clean
Unnamed: 0,,
PDXorax,"The Lump of Labor doesn't apply in this case, ...",lump labor doesnt appli case look rd labor aut...
PrincePizza1,"I think things ""getting better"" to people like...",think thing get better peopl like everybodi ag...
Marmar79,Thank you. This made my dayHe so edgy.\n\nEdit...,thank made dayh edgi edit dont think wrong thi...
IRISHE3,I just wish politicians would put the money sh...,wish politician would put money shit real doll...
RonZiggy,Climate change as well.Wow...just wowWhat is t...,climat chang wellwowjust wowwhat fromwel happe...


In [70]:
df_nlp.shape

(3707, 2)

In [71]:
df_yang_train = pd.read_csv('fin_users_train.csv')

In [72]:
df_yang_test = pd.read_csv('fin_users_test.csv')

In [73]:
df_yang_train.set_index('Unnamed: 0',inplace=True)

In [74]:
df_yang_test.set_index('Unnamed: 0',inplace=True)

In [75]:
df_yang_tr = df_yang_train[df_yang_train['target']==1]

In [76]:
df_yang_ts = df_yang_test[df_yang_test['target']==1]

In [77]:
df_users = df_yang_ts.append(df_yang_tr)

In [47]:
# users = list(df_yang_tr.index)+list(df_yang_ts.index)

In [79]:
df_users.shape

(937, 1)

In [80]:
df_use = pd.concat([df_nlp,df_users],join='inner',axis=1)

In [81]:
df_use.shape

(595, 3)

In [19]:
# import twitter 
import re
import datetime 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation

from gensim import corpora, models
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

np.random.seed(42)

In [89]:
## Try initializing a count vectorizer
stop = list(ENGLISH_STOP_WORDS)
stop.extend(['https','amp'])

cv = CountVectorizer(min_df=10 ,stop_words=stop)


cv_arr = cv.fit_transform(df_use['Clean'])

df_vect = pd.DataFrame(cv_arr.toarray(), columns=cv.get_feature_names())
df_vect.head()

,abandon,abil,abl,abolish,abort,absolut,absurd,abus,aca,acceler,...,youd,youll,young,younger,youth,youtub,youv,yr,zero,zone
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [93]:
texts = [df_vect.columns[df_vect.loc[index,:].nonzero()] for index in df_vect.index]

/Users/yichenh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  """Entry point for launching an IPython kernel.


In [94]:
dictionary = corpora.Dictionary(texts)

In [95]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [96]:
ldamodel = models.ldamodel.LdaModel(corpus,                     # pass in our corpus
                                    id2word = dictionary,       # matches each word to its "number" or "spot" in the dictionary
                                    num_topics = 5,             # number of topics T to find
                                    passes = 5,                 # number of passes through corpus; similar to number of epochs
                                    minimum_probability = 0.01) # only include topics above this probability threshold

In [97]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/yichenh/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.024726 -0.001529       1        1  38.792469
3     -0.013439 -0.009464       2        1  34.048489
1     -0.008705  0.022305       3        1  13.338691
0      0.005691  0.000801       4        1   8.269872
4     -0.008273 -0.012113       5        1   5.550478, topic_info=    Category        Freq     Term       Total  loglift  logprob
53   Default  360.000000     like  360.000000  30.0000  30.0000
68   Default  355.000000    peopl  355.000000  29.0000  29.0000
385  Default  324.000000    think  324.000000  28.0000  28.0000
300  Default  315.000000     make  315.000000  27.0000  27.0000
390  Default  294.000000    trump  294.000000  26.0000  26.0000
83   Default  260.000000    right  260.000000  25.0000  25.0000
198  Default  260.000000   candid  260.000000  24.0000  24.0000
98   Default  279.000000     time  279.000000  23.0000  23.0000
290  Default  279.000000     know  279.000000  22.0000  22.0000
234  Default  335.000000     dont  335.000000  21.0000  21.0000
140  Default  258.000000     good  258.000000  20.0000  20.0000
277  Default  277.000000       im  277.000000  19.0000  19.0000
105  Default  293.000000     want  293.000000  18.0000  18.0000
173  Default  248.000000     vote  248.000000  17.0000  17.0000
355  Default  305.000000      say  305.000000  16.0000  16.0000
62   Default  285.000000     need  285.000000  15.0000  15.0000
109  Default  248.000000     work  248.000000  14.0000  14.0000
394  Default  254.000000      way  254.000000  13.0000  13.0000
72   Default  241.000000   polici  241.000000  12.0000  12.0000
97   Default  287.000000    thing  287.000000  11.0000  11.0000
56   Default  241.000000     look  241.000000  10.0000  10.0000
380  Default  248.000000     talk  248.000000   9.0000   9.0000
345  Default  261.000000   realli  261.000000   8.0000   8.0000
115  Default  258.000000   actual  258.000000   7.0000   7.0000
21   Default  266.000000   doesnt  266.000000   6.0000   6.0000
113  Default  241.000000     year  241.000000   5.0000   5.0000
123  Default  206.000000  countri  206.000000   4.0000   4.0000
379  Default  255.000000  support  255.000000   3.0000   3.0000
77   Default  206.000000   presid  206.000000   2.0000   2.0000
172  Default  252.000000      tri  252.000000   1.0000   1.0000
..       ...         ...      ...         ...      ...      ...
832   Topic5    9.377363    donat   85.769615   0.6779  -6.7264
277   Topic5   22.579451       im  277.494537   0.3825  -5.8477
140   Topic5   21.155586     good  258.527313   0.3882  -5.9128
68    Topic5   26.752687    peopl  355.697571   0.3038  -5.6781
300   Topic5   24.113573     make  315.302246   0.3205  -5.7820
123   Topic5   17.629950  countri  206.133316   0.4324  -6.0951
810   Topic5   16.188042   differ  184.820221   0.4562  -6.1805
367   Topic5   17.477892   someon  211.918518   0.3960  -6.1038
290   Topic5   20.836288     know  279.706726   0.2942  -5.9280
98    Topic5   20.746698     time  279.002960   0.2925  -5.9323
264   Topic5   13.665580    gener  149.872009   0.4964  -6.3498
344   Topic5   14.567805     read  166.039017   0.4579  -6.2859
394   Topic5   19.163097      way  254.132690   0.3064  -6.0117
464   Topic5   16.612593      pay  203.842178   0.3841  -6.1546
521   Topic5   18.206966      lot  235.549255   0.3312  -6.0629
173   Topic5   18.780025     vote  248.855881   0.3072  -6.0319
275   Topic5   16.121752     idea  195.122330   0.3978  -6.1846
52    Topic5   15.848136      let  191.182785   0.4011  -6.2017
115   Topic5   18.852110   actual  258.352631   0.2736  -6.0281
234   Topic5   21.620811     dont  335.256714   0.1500  -5.8911
44    Topic5   17.293682      job  224.472458   0.3279  -6.1144
62    Topic5   19.667885     need  285.584381   0.2157  -5.9857
72    Topic5   17.863773   polici  241.337219   0.2879  -6.0820
380   Topic5   18.126970     talk  248.756165   0.2722  -6.